In [5]:
%run '~/Zhang-Lab/Zhang Lab Code/Boilerplate/model_boilerplate.py'

In [6]:
#Loading models back in 

model_dir = "/home/christianl/Zhang-Lab/Zhang Lab Data/Saved models/Random Forest/Saved_Models_XGBRF_v2"

# Find all saved models; ensure consistent order
model_paths = sorted(glob.glob(os.path.join(model_dir, "target_*.json")))

models = []
for path in model_paths:
    est = xgb.XGBRFRegressor(
        objective='reg:squarederror',
        random_state=42,
        n_estimators=100,
        max_depth=5,
        device='cuda',
        tree_method='hist'
    )
    est.load_model(path)
    models.append(est)

print(f"Loaded {len(models)} models from {model_dir}")


Loaded 16101 models from /home/christianl/Zhang-Lab/Zhang Lab Data/Saved models/Random Forest/Saved_Models_XGBRF_v2


In [ ]:
 # Evaluate model 

# List of estimators (one per target gene) so build predictions matrix 

if isinstance(loaded, list):
    models = loaded
    print("Loaded object is a list of models. n_models:", len(models))
    y_pred = np.column_stack([m.predict(x_test) for m in models])  # (n_samples, n_genes)
else:
    model = loaded
    print("Loaded object type:", type(model))
    # If single-output model but y_test is multi-column, this will raise -- handled later
    y_pred = model.predict(x_test)

print("y_pred.shape:", y_pred.shape)

# diagnostics (safe access)
def safe_attr(obj, name):
    return getattr(obj, name, None) if not isinstance(obj, list) else None

print("model.n_features_in_:", safe_attr(loaded, "n_features_in_"))
print("model.n_outputs_:", safe_attr(loaded, "n_outputs_"))
print("x_test.shape:", x_test.shape)
print("y_test.shape:", y_test.shape)

# Continue with your R^2 / MSE logic expecting y_pred shape (n_samples, n_targets)
from sklearn.metrics import r2_score, mean_squared_error

# If y_pred is 1D, treat as single-output
if y_pred.ndim == 1 or (y_pred.ndim == 2 and y_pred.shape[1] == 1):
    if y_test.ndim == 2 and y_test.shape[1] > 1:
        raise ValueError(
            "Model predicts a single target but y_test contains multiple targets (genes).\n"
            "Select the trained target column before splitting, e.g.:\n"
            "  target = 'GENE_NAME'\n"
            "  y = gene_expression[target]\n"
            "  then redo train_test_split and evaluation."
        )
    y_true = y_test.ravel()
    print("R^2:", r2_score(y_true, y_pred))
    print("MSE:", mean_squared_error(y_true, y_pred))
else:
    print("Multi-output R^2 (uniform_average):", r2_score(y_test, y_pred, multioutput='uniform_average'))
    
    
first = models[0]
print("first estimator type:", type(first))
print("n_features_in_:", getattr(first, "n_features_in_", None))
print("example feature_importances_ (first 10):", getattr(first, "feature_importances_", None)[:10])